In [2]:
import carla, time, pygame, math, random,sys, paho.mqtt.client as mqtt
from enum import Enum

from Adas import Forward_collision_warning_mqtt

sys.path.insert(0,'C:/svs/CARLA_0.9.15/WindowsNoEditor/PythonAPI/carla')
from agents.navigation.basic_agent import BasicAgent

# setup

In [4]:
client = carla.Client("localhost", 2000)
client.set_timeout(50.0)

In [5]:
world = client.get_world()
if not world.get_map().name == 'Carla/Maps/Town03':
    world = client.load_world('Town03')

# Support function

In [6]:
def destroy_all_vehicles(world):
    # Ottieni tutti gli attori nella mappa
    all_actors = world.get_actors()
    
    # Filtra solo i veicoli
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    # Distruggi ciascun veicolo
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [7]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [91]:
def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.brake = 1  # Applica il freno massimo
    vehicle.apply_control(control)
    
def actor_stop_vehicle(vehicle, actor):
    print("STOP")
    control = carla.VehicleControl()
    control.throttle = 0.0  # Nessuna accelerazione
    control.brake = 1.0     # Massima forza frenante
    control.hand_brake = True
    
    vehicle.apply_control(control)

In [9]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 0.8

# Setup vehicles

## Vehicle on a slope

In [10]:
def vehicle_on_a_slope():
    obstacle_vehicle = spawn_vehicle(world, carla.Location(x=152.5, y=-33.249748, z=4), 0, -90) 
    stop_vehicle(obstacle_vehicle)
    return obstacle_vehicle

## Close vehicle on a slope

In [11]:
def close_vehicle_on_a_slope():
    obstacle_vehicle = spawn_vehicle(world, carla.Location(x=151.715500, y=-19.964308, z=3), 0, -90) 
    stop_vehicle(obstacle_vehicle)
    return obstacle_vehicle

## Far away vehicle on a slope

In [12]:
def far_away_vehicle_on_a_slope():
    obstacle_vehicle = spawn_vehicle(world, carla.Location(x=153.095108, y=-58.077873, z=7), 0, -90) 
    stop_vehicle(obstacle_vehicle)
    return obstacle_vehicle

## Forward vehicle

In [68]:
def forward_vehicle():
    obstacle_vehicle = spawn_vehicle(world, carla.Location(x=136.463959, y=-5.8, z=1), 0, 180) 
    return obstacle_vehicle

In [14]:
def stop_vehicle2(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0  # Assicurati che non ci sia accelerazione
    control.steer = 0.0    # Mantieni il volante dritto
    control.brake = 1.0
    vehicle.apply_control(control)

In [15]:
def stop_simulation():
    global run
    run = False
    print("stop")

# Run

In [106]:
destroy_all_vehicles(world)

# vehicle_on_a_slope()

# forward_vehicle()


walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
walker_transform = carla.Transform(carla.Location(x=136.463959, y=-5.8, z=1))
walker = world.spawn_actor(walker_bp, walker_transform)

walker_control = carla.WalkerControl()
walker_control.speed = 1.5  # Velocità in m/s (positivo in avanti)
walker_control.direction = carla.Vector3D(-1, 0, 0)  # Direzione di movimento (x, y, z)


vehicle = spawn_vehicle(world, carla.Location(x=178.759705, y=-5.8, z=1), 2, 180)

world.wait_for_tick()
actor_agent = BasicAgent(vehicle)

actor_agent.set_destination(carla.Location(x=124.759705, y=-5.8, z=1))
# actor_agent.set_destination(carla.Location(x=153.095108, y=-58.077873, z=7))
actor_agent.set_target_speed(60)
actor_agent.ignore_vehicles()


try:
    run = True
    throttle = 0.7

    adas = Forward_collision_warning_mqtt(
        world = world,
        min_ttc = 0.05,
        attached_vehicle = vehicle,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation()
    )

    while run:
        vehicle.apply_control(actor_agent.run_step())
        walker.apply_control(walker_control)
        world.wait_for_tick()
        

#     while True:
    actor_stop_vehicle(vehicle, actor_agent)
#         world.wait_for_tick()
        
#     adas.destroy()
#     vehicle.destroy()
    time.sleep(10)

finally:
    print("done")
    adas.destroy()
    vehicle.destroy()

Numero di veicoli trovati: 0
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
stop
STOP
done


RuntimeError: time-out of 50000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [105]:
# Ottieni tutti gli attori nel mondo
all_actors = world.get_actors()

# Filtra solo i pedoni
pedestrians = all_actors.filter('walker.pedestrian.*')

print(f"Numero di pedoni trovati: {len(pedestrians)}")

# Distruggi ciascun pedone
for pedestrian in pedestrians:
    pedestrian.destroy()
    print(f"Pedone distrutto: {pedestrian.id}")


Numero di pedoni trovati: 1
Pedone distrutto: 232
